# 7 - Functions

## 7.1. Writing Functions That Accept Any Number of Arguments

In [1]:
def avg(first, *rest):
    return (first + sum(rest)) / (1 + len(rest))


In [2]:
avg(1, 2)

1.5

In [3]:
avg(1, 2, 3)

2.0

To accept any number of keyword arguments, use an argument that starts with **.

In [4]:
import html

def make_element(name, value, **attrs):
    keyvals = [' %s="%s"' % item for item in attrs.items()]
    attr_str = ''.join(keyvals)
    element = '<{name}{attrs}>{value}</{name}>'.format(
        name=name,
        attrs=attr_str,
        value=html.escape(value))
    return element


In [5]:
make_element('item', 'Albatross', size='large', quantity=6)

'<item size="large" quantity="6">Albatross</item>'

In [6]:
make_element('p', '<spam>')

'<p>&lt;spam&gt;</p>'

In [8]:
# generally
def anyargs(*args, **kwargs):
    print(args, type(args))     # tuple
    print(kwargs, type(kwargs)) # dict

anyargs("arg1", "arg2", kwarg1="kwarg1", kwarg2="kwarg2")

('arg1', 'arg2') <class 'tuple'>
{'kwarg1': 'kwarg1', 'kwarg2': 'kwarg2'} <class 'dict'>


## Writing Functions That Only Accept Keyword Arguments
Easy to implement if you place the keyword arguments after a * argument or a single unnamed *.

In [10]:
def recv(maxsize, *, block):
    print(block)

recv(1024, block=True)

True


In [11]:
recv(1024, True)

TypeError: recv() takes 1 positional argument but 2 were given

Keyword-only arguments are often a good way to enforce greater code clarity.

## Attaching Informational Metadata to Function Arguments
They are not type checks, nor do they make Python behave any differently than it did before. However, they might give useful hints to others reading the source code about what you had in mind.

In [12]:
def add(x:int, y:int) -> int:
    return x + y


In [13]:
help(add)

Help on function add in module __main__:

add(x:int, y:int) -> int



In [14]:
add.__annotations__

{'x': int, 'y': int, 'return': int}

## Returning Multiple Values from a Function

In [15]:
def func():
    return 1, 2, 3

func()

(1, 2, 3)

## Defining Functions with Default Arguments

In [16]:
def func(a, b=3):
    return a + b


In [17]:
func(2)

5

In [18]:
func(3, 5)

8

## Defining Anonymous or Inline Functions

In [20]:
add = lambda x, y: x + y
add

<function __main__.<lambda>(x, y)>

In [21]:
add(1, 2)

3

## Capturing Variables in Anonymous Functions

In [22]:
x = 10
a = lambda y: x + y
a(10)

20

In [23]:
x = 20
b = lambda y: x + y
b(10)

30

In [24]:
a(10), b(10)

(30, 30)

The value of x in the lambda expressions is whatever the value of the x variable happens to be at the time of execution.

## Making an N-Argument Callable Work As a Callable with Fewer Arguments